## Import libraries

In [2]:
import pandas as pd 
import numpy as np 

## Import data files

In [3]:
## Read general meadian file
global_median = pd.read_csv('/Users/adityaponnada/Downloads/time_study_data/general_rnn_medians.csv')

In [7]:
global_median.head()
print(global_median.columns)
print(global_median.shape)

Index(['Unnamed: 0', 'is_weekend', 'in_battery_saver_mode', 'charging_status',
       'screen_on', 'dist_from_home', 'is_phone_locked', 'last_phone_usage',
       'closeness_to_sleep_time', 'closeness_to_wake_time', 'mims_5min',
       'days_in_study', 'completion_24h', 'completion_1h',
       'time_between_prompts', 'time_since_last_answered',
       'completion_since_wake', 'completion_since_start',
       'time_of_day_Afternoon', 'time_of_day_Early Morning',
       'time_of_day_Evening', 'time_of_day_Late Night', 'time_of_day_Morning',
       'time_of_day_Night', 'location_category_Home',
       'location_category_Other', 'location_category_School',
       'location_category_Transit', 'location_category_Work',
       'wake_day_part_0.0', 'wake_day_part_1.0', 'wake_day_part_2.0',
       'wake_day_part_3.0'],
      dtype='object')
(1, 33)


In [10]:
## Read the global means file
global_means = pd.read_csv('/Users/adityaponnada/Downloads/time_study_data/global_means_general_rnn.csv')
global_means.head()
print(global_means.columns)
print(global_means.shape)

Index(['Unnamed: 0', 'dist_from_home', 'last_phone_usage',
       'closeness_to_sleep_time', 'closeness_to_wake_time', 'mims_5min',
       'completion_24h', 'completion_1h', 'time_between_prompts',
       'time_since_last_answered', 'completion_since_wake',
       'completion_since_start'],
      dtype='object')
(1, 12)


In [11]:
## Read the withdrew processed features file
withdrew_features = pd.read_csv('/Users/adityaponnada/Downloads/time_study_data/processed_features_withdrew.csv')
print(withdrew_features.shape)
withdrew_features.head()

(235071, 66)


,participant_id,outcome,is_weekend,in_battery_saver_mode,charging_status,screen_on,dist_from_home,is_phone_locked,last_phone_usage,closeness_to_sleep_time,...,mi_time_of_day_Night,mi_location_category_Home,mi_location_category_Other,mi_location_category_School,mi_location_category_Transit,mi_location_category_Work,mi_wake_day_part_0.0,mi_wake_day_part_1.0,mi_wake_day_part_2.0,mi_wake_day_part_3.0
0,ambushdollhousegenerous@timestudy_com,0,0,0.0,1.0,0,0.060436,1.0,11.1,980.950000,...,0,0,0,0,0,0,0,0,0,0
1,ambushdollhousegenerous@timestudy_com,1,0,0.0,0.0,1,0.059622,0.0,0.0,830.900000,...,0,0,0,0,0,0,0,0,0,0
2,ambushdollhousegenerous@timestudy_com,1,0,NaN,NaN,1,0.042405,0.0,0.0,487.783333,...,0,0,0,0,0,0,0,0,0,0
3,ambushdollhousegenerous@timestudy_com,0,0,0.0,1.0,1,0.008069,0.0,0.0,242.950000,...,0,0,0,0,0,0,0,0,0,0
4,ambushdollhousegenerous@timestudy_com,1,0,0.0,0.0,1,0.059189,0.0,0.0,186.583333,...,0,0,0,0,0,0,0,0,0,0


In [12]:
withdrew_features.columns

Index(['participant_id', 'outcome', 'is_weekend', 'in_battery_saver_mode',
       'charging_status', 'screen_on', 'dist_from_home', 'is_phone_locked',
       'last_phone_usage', 'closeness_to_sleep_time', 'closeness_to_wake_time',
       'mims_5min', 'days_in_study', 'completion_24h', 'completion_1h',
       'time_between_prompts', 'time_since_last_answered',
       'completion_since_wake', 'completion_since_start',
       'time_of_day_Afternoon', 'time_of_day_Early Morning',
       'time_of_day_Evening', 'time_of_day_Late Night', 'time_of_day_Morning',
       'time_of_day_Night', 'location_category_Home',
       'location_category_Other', 'location_category_School',
       'location_category_Transit', 'location_category_Work',
       'wake_day_part_0.0', 'wake_day_part_1.0', 'wake_day_part_2.0',
       'wake_day_part_3.0', 'mi_is_weekend', 'mi_in_battery_saver_mode',
       'mi_charging_status', 'mi_screen_on', 'mi_dist_from_home',
       'mi_is_phone_locked', 'mi_last_phone_usage',
 

## Impute missing data
Using fillforward + global median

In [13]:
def impute_within_participant(withdrew_df, global_median, id_col='participant_id'):
    """Impute missing values by forward-filling within each participant.

    - Skips columns starting with "mi_".
    - Excludes `id_col` and outcome columns (`outcome`, `outcomes`).
    - If the first observation for a participant is NaN, fills it using `global_median`.

    Args:
        withdrew_df (pd.DataFrame): input features dataframe (observations ordered per participant).
        global_median (pd.Series or pd.DataFrame): mapping of column -> median value.
        id_col (str): participant id column name (default: 'participant_id').

    Returns:
        pd.DataFrame: imputed copy of `withdrew_df`.
    """
    import pandas as pd
    import numpy as np

    if id_col not in withdrew_df.columns:
        raise ValueError(f"id_col '{id_col}' not found in withdrew_df columns")

    df = withdrew_df.copy()

    # Exclude participant id and outcome columns (case-insensitive)
    exclude = {id_col.lower(), 'outcome', 'outcomes'}

    # Build list of columns to impute
    cols_to_impute = [c for c in df.columns if (c.lower() not in exclude) and (not c.lower().startswith('mi_'))]

    # Forward-fill within each participant
    try:
        df[cols_to_impute] = df.groupby(id_col, sort=False)[cols_to_impute].ffill()
    except Exception:
        # Fallback if grouping/ffill fails for mixed dtypes
        df[cols_to_impute] = df.groupby(id_col, sort=False)[cols_to_impute].apply(lambda g: g.ffill())

    # Helper to extract median from global_median (Series or DataFrame)
    def _get_global_median(col):
        try:
            if isinstance(global_median, pd.Series):
                return global_median.get(col, np.nan)
            if isinstance(global_median, pd.DataFrame):
                # If column exists as DataFrame column, take its first non-null
                if col in global_median.columns:
                    vals = global_median[col].dropna().values
                    if len(vals) > 0:
                        return vals[0]
                # If index contains column names and there's a 'median' column
                if 'median' in global_median.columns and col in global_median.index:
                    return global_median.loc[col, 'median']
                # If index contains column names, return first non-null in that row
                if col in global_median.index:
                    row = global_median.loc[col]
                    if hasattr(row, 'dropna'):
                        vals = row.dropna().values
                        if len(vals) > 0:
                            return vals[0]
        except Exception:
            pass
        return np.nan

    # Fill remaining NaNs (leading NaNs) with global median (or df median as fallback)
    for col in cols_to_impute:
        if df[col].isna().any():
            med = _get_global_median(col)
            if pd.isna(med):
                try:
                    med = df[col].median(skipna=True)
                except Exception:
                    med = np.nan
            if pd.notna(med):
                df[col] = df[col].fillna(med)

    return df


In [14]:
withdrew_features = impute_within_participant(withdrew_features, global_median, id_col='participant_id')
withdrew_features.columns
withdrew_features.head()

,participant_id,outcome,is_weekend,in_battery_saver_mode,charging_status,screen_on,dist_from_home,is_phone_locked,last_phone_usage,closeness_to_sleep_time,...,mi_time_of_day_Night,mi_location_category_Home,mi_location_category_Other,mi_location_category_School,mi_location_category_Transit,mi_location_category_Work,mi_wake_day_part_0.0,mi_wake_day_part_1.0,mi_wake_day_part_2.0,mi_wake_day_part_3.0
0,ambushdollhousegenerous@timestudy_com,0,0,0.0,1.0,0,0.060436,1.0,11.1,980.950000,...,0,0,0,0,0,0,0,0,0,0
1,ambushdollhousegenerous@timestudy_com,1,0,0.0,0.0,1,0.059622,0.0,0.0,830.900000,...,0,0,0,0,0,0,0,0,0,0
2,ambushdollhousegenerous@timestudy_com,1,0,0.0,0.0,1,0.042405,0.0,0.0,487.783333,...,0,0,0,0,0,0,0,0,0,0
3,ambushdollhousegenerous@timestudy_com,0,0,0.0,1.0,1,0.008069,0.0,0.0,242.950000,...,0,0,0,0,0,0,0,0,0,0
4,ambushdollhousegenerous@timestudy_com,1,0,0.0,0.0,1,0.059189,0.0,0.0,186.583333,...,0,0,0,0,0,0,0,0,0,0
